# Cross-validated logL from HMM object

In [4]:
import sys
import re
from contextlib import redirect_stderr
from io import StringIO
from tqdm import tqdm
import os
import glob
import h5py
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from hmmlearn import hmm

In [5]:
# ─── Settings ────────────────────────────────────────────────────────────────────
INPUT_DIR     = "/home/jovyan/narratives-project/shirer_components"
DATASET       = "timeseries"
STATE_COUNTS  = [10, 14]
N_SPLITS      = 5
MAX_ITER      = 500
RANDOM_SEED   = 42
# ─────────────────────────────────────────────────────────────────────────────────

def load_data(input_dir):
    data = []
    for path in sorted(glob.glob(os.path.join(input_dir, "*.h5"))):
        with h5py.File(path, "r") as f:
            ts = f[DATASET][()].T
        ts = StandardScaler().fit_transform(ts)
        data.append(ts)
    return data

def compute_cv_loglik(data, k, n_splits=N_SPLITS):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    scores = []

    for fold, (train_idx, test_idx) in enumerate(kf.split(data), 1):
        train = [data[i] for i in train_idx]
        test  = [data[i] for i in test_idx]

        X_train = np.vstack(train)
        lengths_train = [d.shape[0] for d in train]

        print(f"\n  [Fold {fold}] Fitting HMM (k={k}) with verbose EM output:")
        model = hmm.GaussianHMM(n_components=k, covariance_type='full', n_iter=MAX_ITER, verbose=True)
        model.fit(X_train, lengths_train)

        X_test = np.vstack(test)
        lengths_test = [d.shape[0] for d in test]
        try:
            logL = model.score(X_test, lengths_test)
            scores.append(logL)
            print(f"  [Fold {fold}] logL = {logL:.1f}")
        except Exception as e:
            print(f"  [ERROR] Fold {fold} for k={k}: {e}")

    return np.mean(scores), np.std(scores)

def main():
    data = load_data(INPUT_DIR)
    print(f"✔ Loaded {len(data)} subjects")

    cv_results = []

    for k in STATE_COUNTS:
        print(f"\n=== Running CV for k={k} ===")
        mean_cvll, std_cvll = compute_cv_loglik(data, k)
        print(f"→ k={k}: Mean CVLL = {mean_cvll:.2f}, SD = {std_cvll:.2f}")
        cv_results.append({
            "k": k,
            "mean_cvll": mean_cvll,
            "std_cvll": std_cvll
        })

    # Sort and report best
    cv_results.sort(key=lambda x: x["mean_cvll"], reverse=True)
    print("\n=== Summary ===")
    for res in cv_results:
        print(f"k={res['k']:>2} | CVLL = {res['mean_cvll']:.2f} ± {res['std_cvll']:.2f}")

    best = cv_results[0]
    print(f"\n✅ Best model: k={best['k']} (CVLL = {best['mean_cvll']:.2f})")

if __name__ == "__main__":
    main()

✔ Loaded 75 subjects

=== Running CV for k=10 ===

  [Fold 1] Fitting HMM (k=10) with verbose EM output:


         1 -203903.04903179             +nan
         2 -194940.36320081   +8962.68583098
         3 -190116.54762887   +4823.81557195
         4 -187293.95693624   +2822.59069263
         5 -185731.32527972   +1562.63165652
         6 -184567.39696463   +1163.92831509
         7 -183652.10559544    +915.29136919
         8 -182922.11534528    +729.99025016
         9 -182332.61708124    +589.49826404
        10 -181857.70104805    +474.91603318
        11 -181465.45469748    +392.24635057
        12 -181116.63077082    +348.82392666
        13 -180774.02560360    +342.60516721
        14 -180524.28207249    +249.74353112
        15 -180343.33601562    +180.94605687
        16 -180224.62118144    +118.71483418
        17 -180138.15241336     +86.46876808
        18 -180073.51962185     +64.63279151
        19 -180024.18931378     +49.33030807
        20 -179983.77110268     +40.41821110
        21 -179941.11893609     +42.65216658
        22 -179885.44363742     +55.67529868
        23

  [Fold 1] logL = -48354.1

  [Fold 2] Fitting HMM (k=10) with verbose EM output:


         1 -205404.79379304             +nan
         2 -195049.69507165  +10355.09872139
         3 -190726.07149757   +4323.62357408
         4 -187840.13229471   +2885.93920287
         5 -186056.48308546   +1783.64920925
         6 -184754.04146352   +1302.44162193
         7 -183829.12390028    +924.91756325
         8 -183141.63998486    +687.48391542
         9 -182507.34970553    +634.29027933
        10 -181797.85038373    +709.49932181
        11 -180946.51192467    +851.33845906
        12 -180275.54997050    +670.96195417
        13 -179861.99444603    +413.55552447
        14 -179592.47885875    +269.51558728
        15 -179395.29578606    +197.18307269
        16 -179233.11876891    +162.17701715
        17 -179097.68642585    +135.43234305
        18 -178979.08287009    +118.60355576
        19 -178885.06257243     +94.02029766
        20 -178817.80608180     +67.25649063
        21 -178761.76941288     +56.03666892
        22 -178703.11601679     +58.65339610
        23

  [Fold 2] logL = -46650.9

  [Fold 3] Fitting HMM (k=10) with verbose EM output:


...

         1 -212275.57676071             +nan
         2 -197777.44202843  +14498.13473228
         3 -192996.14775964   +4781.29426879
         4 -190575.03764117   +2421.11011847
         5 -188985.02024888   +1590.01739228
         6 -187833.32657498   +1151.69367391
         7 -186839.15419612    +994.17237886
         8 -185884.06349210    +955.09070402
         9 -184973.10575981    +910.95773229
        10 -184218.68486610    +754.42089371
        11 -183606.06457612    +612.62028998
        12 -183102.51748830    +503.54708782
        13 -182699.52584378    +402.99164452
        14 -182363.20369394    +336.32214984
        15 -182097.54823760    +265.65545634
        16 -181851.68670829    +245.86152931
        17 -181662.02321539    +189.66349291
        18 -181500.49706312    +161.52615227
        19 -181353.47991774    +147.01714537
        20 -181219.95857815    +133.52133960
        21 -181111.55673551    +108.40184263
        22 -181029.23084080     +82.32589471
        23

  [Fold 4] logL = -41755.9

  [Fold 5] Fitting HMM (k=14) with verbose EM output:


         1 -212340.76372195             +nan
         2 -196609.34342136  +15731.42030059
         3 -193367.52390911   +3241.81951224
         4 -190482.76877348   +2884.75513563
         5 -188227.20496554   +2255.56380794
         6 -186497.08507363   +1730.11989191
         7 -185251.97619537   +1245.10887825
         8 -184215.90604538   +1036.07014999
         9 -183411.67763110    +804.22841429
        10 -182733.28982577    +678.38780533
        11 -182166.98691388    +566.30291189
        12 -181703.49106301    +463.49585088
        13 -181334.97988370    +368.51117930
        14 -180982.34251623    +352.63736747
        15 -180740.09698271    +242.24553353
        16 -180543.71900628    +196.37797643
        17 -180392.71161911    +151.00738717
        18 -180270.96943499    +121.74218412
        19 -180171.76694529     +99.20248970
        20 -180069.53989890    +102.22704638
        21 -179970.74085390     +98.79904500
        22 -179877.63391003     +93.10694387
        23

  [Fold 5] logL = -44901.7
→ k=14: Mean CVLL = -45803.31, SD = 2331.69

=== Summary ===
k=14 | CVLL = -45803.31 ± 2331.69
k=10 | CVLL = -46145.43 ± 2151.04

✅ Best model: k=14 (CVLL = -45803.31)


       309 -176243.59397693      +0.00967549
